<a href="https://colab.research.google.com/github/Oluwajuwon-O/sentiment_analysis/blob/main/getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [ ]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())

    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews


In [ ]:
df = pd.DataFrame(reviews, columns=['reviews'])
df.head()

,reviews
0,✅ Trip Verified | Check in fast and friendly....
1,✅ Trip Verified | This is the first time I ha...
2,✅ Trip Verified | Flew business class from Do...
3,✅ Trip Verified | Starting off at Heathrow Te...
4,Not Verified | We have flown this route with ...


In [ ]:
df.to_csv("BA_reviews.csv", index=False)

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [ ]:
reviews = pd.read_csv('BA_reviews.csv')
reviews.head()

,reviews
0,✅ Trip Verified | Check in fast and friendly....
1,✅ Trip Verified | This is the first time I ha...
2,✅ Trip Verified | Flew business class from Do...
3,✅ Trip Verified | Starting off at Heathrow Te...
4,Not Verified | We have flown this route with ...


In [ ]:
reviews_df = reviews.copy()

reviews_df

,reviews
0,✅ Trip Verified | Check in fast and friendly....
1,✅ Trip Verified | This is the first time I ha...
2,✅ Trip Verified | Flew business class from Do...
3,✅ Trip Verified | Starting off at Heathrow Te...
4,Not Verified | We have flown this route with ...
...,...
995,✅ Trip Verified | London Heathrow to Houston....
996,Not Verified | Larnaca to London. This A320 i...
997,✅ Trip Verified | I booked my BA flight from ...
998,✅ Trip Verified | The booking process was eas...



* ✅ is pointless
* leading spaces
* | is not supposed to be there
* trip verification can be in separate column
* \r\n in some entries

* remove ✅
* split by |
* remove leading spaces
* remove \r\n

In [ ]:
reviews_df = pd.DataFrame(reviews_df.reviews.str.replace('✅','').replace('\r\n',''))

reviews_df[['verification','review']] = reviews_df['reviews'].str.split('|', expand=True)

reviews_df.drop('reviews',inplace = True,axis = 'columns')

In [ ]:
reviews_df = pd.DataFrame(reviews_df['review'].str.strip())
reviews_df

,review
0,Check in fast and friendly. Security a breeze....
1,This is the first time I have seen the new Clu...
2,Flew business class from Doha to London on 31s...
3,Starting off at Heathrow Terminal 3. Check in ...
4,We have flown this route with EasyJet regularl...
...,...
995,London Heathrow to Houston. Despite sitting at...
996,Larnaca to London. This A320 is totally unsuit...
997,I booked my BA flight from Zurich to London He...
998,The booking process was easy enough but they h...


In [ ]:
from transformers import pipeline

In [ ]:
trans_pipeline = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
trans_pipeline(reviews_df['review'][0])

[{'label': 'POSITIVE', 'score': 0.995097815990448}]

In [ ]:
trans_pipeline(reviews_df['review'][4])

[{'label': 'NEGATIVE', 'score': 0.9963898062705994}]

In [ ]:
type(reviews_df.review.values)

numpy.ndarray

In [ ]:
for i in reviews_df['review'][:10]:
  print(i+'\n')

Check in fast and friendly. Security a breeze. Lounge was busy early evening but comfortable and clean. Flight attendants welcoming. Seat a nightmare it was the reverse/forward with a step over from the window seat, who the hell thought that was a good idea?? Meal were OK but all curry based, like curry and I was on a flight to India so. FA was brilliant as the only flyer awake on an overnight flight. Improving since my last BA flight

This is the first time I have seen the new Club World suite. The seat was comfortable but unlike other airlines, BA has crammed as many seats in business class as is humanly possible so the cabin felt cramped. The crew behaved as though we, the passengers, should have been grateful for them turning up for their shift. They looked scruffy in the new uniforms, clearly the dress code policy has been scrapped. I appreciate that appearance is no measure of service quality (in the UK alone) but the service was appalling. The menu lacked choice and the food was

In [ ]:
trans_pipeline(reviews_df['review'].values)

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).